In [1]:
import numpy as np
from copy import deepcopy
import pygame
from game import Game
import time
from GUI import *
import random

pygame 2.1.2 (SDL 2.0.18, Python 3.10.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

import math

# Weight and bias dimensions for the NN
first_layer_hidden_weights = (32,90) #32,90
first_layer_hidden_bias = (1,90) #90
second_layer_hidden_weights = (90,40) #90,40 
second_layer_hidden_bias = (1,40) #40
third_layer_hidden_weights = (40,10) #40,10
third_layer_hidden_bias = (1,10) #10

class Evol_Player(object):
    def __init__(self, number, first_layer_weights, first_layer_bias, second_layer_weights, second_layer_bias, third_layer_weights, third_layer_bias):
        self.number = number
        self.score = 0
        self.first_layer_weights = first_layer_weights
        self.first_layer_bias = first_layer_bias
        self.second_layer_weights = second_layer_weights
        self.second_layer_bias = second_layer_bias
        self.third_layer_weights = third_layer_weights
        self.third_layer_bias = third_layer_bias

        self.win = 0
        self.loss = 0
        self.draw = 0

    def getWeights(self):
        return self.first_layer_weights, self.first_layer_bias,  self.second_layer_weights, self.second_layer_bias,  self.third_layer_weights,  self.third_layer_bias

# Used in the NN
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def evolutionary_player(count):
    
    first_layer_weights = np.random.normal(0, scale=1.0, size=first_layer_hidden_weights)
    first_layer_bias = np.random.normal(0, scale=1.0, size=first_layer_hidden_bias)
    second_layer_weights = np.random.normal(0, scale=1.0, size=second_layer_hidden_weights)
    second_layer_bias = np.random.normal(0, scale=1.0, size=second_layer_hidden_bias)
    third_layer_weights = np.random.normal(0, scale=1.0, size=third_layer_hidden_weights)
    third_layer_bias = np.random.normal(0, scale=1.0, size=third_layer_hidden_bias)

    return Evol_Player(count, first_layer_weights, first_layer_bias, second_layer_weights, second_layer_bias, third_layer_weights, third_layer_bias)

# Use this Neural Network as the heuristic function for the minimax tree
def predict_nn(board, player):

    first_hidden_output = tanh(np.dot(board, player.first_layer_weights) + player.first_layer_bias)
    second_hidden_output = tanh(np.dot(first_hidden_output, player.second_layer_weights) + player.second_layer_bias)
    third_layer_output = tanh(np.dot(second_hidden_output, player.third_layer_weights) + player.third_layer_bias)

    output = np.sum(third_layer_output)
    
    return output

def createNeuralNetwork(offspring1AfterMutation, offspring2AfterMutation, count):

    first_layer_bias1, second_layer_bias1, third_layer_bias1, first_layer_weights1, second_layer_weights1, third_layer_weights1 = offspring1AfterMutation
    first_layer_bias2, second_layer_bias2, third_layer_bias2, first_layer_weights2, second_layer_weights2, third_layer_weights2 = offspring2AfterMutation

    player1 = Evol_Player(count + 1, first_layer_weights1, first_layer_bias1, second_layer_weights1, second_layer_bias1, third_layer_weights1, third_layer_bias1)
    player2 = Evol_Player(count + 2, first_layer_weights2, first_layer_bias2, second_layer_weights2, second_layer_bias2, third_layer_weights2, third_layer_bias2)

    return player1, player2


In [3]:
def simulate_move(piece, move, board, game, skip):
    """
    Simulates a move on the board by moving a piece, capturing any opponent's piece, and updating the game state.
    and return updated game board after the move.
    """
    board.move(piece, move[0], move[1])  # Move the piece on the board.
    if skip:
        board.remove(skip)  # Remove opponent's piece that was captured during the move.

    return board

In [4]:
def get_all_moves(board, color, game):
    """
    Gets all valid moves for a given color on the current game board.
    and returns a List of all possible game boards after making each move.
    """
    moves = []

    for piece in board.get_all_pieces(color):
        valid_moves = board.get_valid_moves(piece)  # Get all valid moves for the piece.
        for move, skip in valid_moves.items():
            temp_board = deepcopy(board)
            temp_piece = temp_board.get_piece(piece.row, piece.col)
            new_board = simulate_move(temp_piece, move, temp_board, game, skip)  # Simulate the move on the board.
            moves.append(new_board)
    
    return moves

In [5]:
def alpha_beta(position, depth, alpha, beta, max_player, game, human_play):
    """
    Implementation of the Alpha-Beta Pruning algorithm over minimax
    """
    if depth == 0:
        # If depth is 0, evaluate the current position and return the score along with the position.
        return position.evaluate(game.board.player), position

    if max_player:
        if human_play == True: 
            if game.turn == "red":
                color_m = "red"  # Determine the color of the opponent's pieces
            else:
                color_m = "white"  # Determine the color of the own pieces
        else:
            if game.turn == "red":
                color_m = "white" 
            else:
                color_m = "red"  
        best_move = None
        for move in get_all_moves(position, color_m, game):
            # Recursively call alpha_beta with the next move and update alpha value.
            evaluation = alpha_beta(move, depth-1, alpha, beta, False, game, human_play)[0]
            if evaluation > alpha:
                alpha = evaluation
                best_move = move
            if alpha >= beta:
                # Perform pruning if alpha is greater than or equal to beta.
                break

        return alpha, best_move
    else:
        best_move = None
        for move in get_all_moves(position, game.turn, game):
            # Recursively call alpha_beta with the next move and update beta value.
            evaluation = alpha_beta(move, depth-1, alpha, beta, True, game, human_play)[0]
            if evaluation < beta:
                beta = evaluation
                best_move = move
            if beta <= alpha:
                # Perform pruning if beta is less than or equal to alpha.
                break

        return beta, best_move

In [6]:
#INITIALLY CHECKING THE ALGORITHM USING THIS
def minimax(position, depth, max_player, game, human_play):
    """
    Implementation of the minimax algorithm for game AI.
    """
    if depth == 0:
        return position.evaluate(game.board.player), position  # Return evaluation of current position and the position itself
    
    if max_player:
        maxEval = float('-inf')  #
        best_move = None 
        if human_play == True: 
            if game.turn == "red":
                color_m = "red"  # Determine the color of the opponent's pieces
            else:
                color_m = "white"  # Determine the color of the own pieces
        else:
            if game.turn == "red":
                color_m = "white" 
            else:
                color_m = "red"  
        for move in get_all_moves(position, color_m, game): 
            evaluation = minimax(move, depth-1, False, game, human_play)[0] 
            maxEval = max(maxEval, evaluation)  # Update the maximum evaluation score
            if maxEval == evaluation:
                best_move = move  # Update the best move if a better move is found
        
        return maxEval, best_move  
    else:
        minEval = float('inf')  
        best_move = None  
        for move in get_all_moves(position, game.turn, game):  
            evaluation = minimax(move, depth-1, True, game, human_play)[0]  
            minEval = min(minEval, evaluation)  # Update the minimum evaluation score
            if minEval == evaluation:
                best_move = move  
        
        return minEval, best_move  # Return the minimum evaluation score and the best move found

In [7]:
#HUMAN VS AI
def human_AI(play1):
    player1 = play1
    player2 = None
    obj = Game(player1, player2)
    start_time = time.time()
    counter = 0
    
    while counter < 40:
        
        old_pieces = obj.board.red_left + obj.board.white_left
        if obj.turn == "red":
            value, new_board = alpha_beta(obj.get_board(), 2, float("-inf"), float("inf"), obj.turn, obj, True)
            #value, new_board = minimax(obj.get_board(), 2, obj.turn, obj, True)
            obj.ai_move(new_board)
            print(new_board.board)
            gui = Checkerboard(obj.board.board)

        else:
            gui = Checkerboard(obj.board.board)
            row, col = gui.run()
            obj.select(row, col)
            
            
        print(obj.turn)
        new_pieces = obj.board.red_left + obj.board.white_left
        difference = old_pieces - new_pieces
        if difference > 0:
            counter = 0
        else:
            counter += 1
        print("DIFF: ", old_pieces - new_pieces)
        winner = obj.winner()

        if winner == "red":
            best_player = obj.player1
        else:
            best_player = obj.player2

    end_time = time.time()
    print("Duration: ", end_time - start_time)

    return new_board, winner, best_player

In [8]:
def AI_vs_AI(player_1, player_2):
    player1 = player_1
    player2 = player_2
    obj = Game(player1, player2)
    start_time = time.time()
    counter = 0
    while counter < 40:
        old_pieces = obj.board.red_left + obj.board.white_left
        # old_pieces = len(obj.board.get_all_pieces(obj.turn)) + len(obj.board.get_all_pieces(opponent))
        value, new_board = alpha_beta(obj.get_board(), 2, float("-inf"), float("inf"), obj.turn, obj, False)
        #value, new_board = minimax(obj.get_board(), 2, obj.turn, obj, False)
        #print(obj.turn)
        # print(new_board.board)
        obj.ai_move(new_board)
        new_pieces = obj.board.red_left + obj.board.white_left
        difference = old_pieces - new_pieces
        if difference > 0:
            counter = 0
        else:
            counter += 1
        # print(counter)
        # print("DIFF: ", old_pieces - new_pieces)
        winner = obj.winner()

        if winner == "white":
            best_player = obj.player1
        else:
            best_player = obj.player2

    end_time = time.time()
    #print("Duration: ", end_time - start_time)

    return new_board, winner, best_player

In [9]:
def fitnessForBinaryTournament(parent_1, parent_2, chromosomes):

    

    newchromosomes = chromosomes.copy()
    #chromosomes.remove(parent_1)
    #chromosomes.remove(parent_2)

    newchromosomes.remove(parent_1)
    newchromosomes.remove(parent_2)
    
    parent1Score = 0
    parent2Score = 0

    randomPlayers = np.random.choice(newchromosomes, size=3, replace=False)
   
    for player2 in randomPlayers:
        print("match start")
        obj = Game(parent_1, player2)
        new_board, winner, best_player = AI_vs_AI(parent_1, player2)

        # if winner=="draw":
        #     parent1Score += 1

        #     player2.draw += 1
        #     parent_1.draw += 1

        #     parent_1.score += 1
        #     player2.score += 1

        # else:
        if winner == 'white':
            # print("bnum: ",best_player.number)
            # parent1Score -= 1
            # parent_1.loss += 1
            # player2.win += 1
            # parent_1.score -= 1
            # player2.score += 2
            parent_1.win += 1
            player2.loss += 1

        else:
            # parent1Score += 2
            # parent_1.win += 1
            # player2.loss += 1
            # parent_1.score += 2
            # player2.score -= 1
            player2.win += 1
            parent_1.loss += 1
        
        player2.score = player2.win - player2.loss
        parent_1.score = parent_1.win - parent_1.loss
        parent1Score = parent_1.score

    for player2 in randomPlayers:
        obj = Game(parent_2, player2)
        new_board, winner, best_player = AI_vs_AI(parent_2, player2)

        # if winner=="draw":
        #     parent2Score += 1

        #     parent_2.draw += 1
        #     player2.draw += 1

        #     parent_2.score += 1
        #     player2.score += 1

        # else:
        if winner == 'white':
            # print("winner", winner, best_player)
            
            # print(parent_2)
            # parent2Score -= 1
            # parent_2.loss += 1
            # player2.win += 1
            # parent_2.score -= 1
            # player2.score += 2
            parent_2.win += 1
            player2.loss += 1
        else:
            # parent2Score += 2
            # player2.loss += 1
            # parent_2.win += 1
            # parent_2.score += 2
            # player2.score -= 1
            player2.win += 1
            parent_2.loss += 1
        
        player2.score = player2.win - player2.loss
        parent_2.score = parent_2.win - parent_2.loss
        parent2Score = parent_2.score
                


    if parent2Score >= parent1Score:
        return parent_2
    else:
        return parent_1

    

In [10]:
def parentSelection(chromosomes, type):

    if type == "random":
        index = np.random.choice(chromosomes, 2, replace=False) 
        
        parent1 = index[0]
        parent2 = index[1]

        if parent1 == parent2:
            print("Parents are the same??")

 
    elif type == "binary_tournament":

        nchromosomes = chromosomes.copy()

        index = np.random.choice(nchromosomes, 2, replace=False) 

        parent_1 = index[0]
        parent_2 = index[1]
       
        # Choose the fittest chromosome as the first parent
        print("Tournament for Parent 1")
        parent1 = fitnessForBinaryTournament(parent_1, parent_2, nchromosomes)
        
        nchromosomes.remove(parent1)
        
        index = np.random.choice(nchromosomes, 2, replace=False) 
        # repeats for second parent
        parent_1 = index[0]
        parent_2 = index[1]
        
        parent2 = fitnessForBinaryTournament(parent_1, parent_2, nchromosomes)


    return parent1, parent2

In [11]:
def crossover(parent1, parent2):

    p1first_layer_weights, p1first_layer_bias, p1second_layer_weights, p1second_layer_bias, p1third_layer_weights,  p1third_layer_bias = parent1.getWeights()
    p2first_layer_weights, p2first_layer_bias, p2second_layer_weights, p2second_layer_bias, p2third_layer_weights,  p2third_layer_bias = parent2.getWeights()
    
    crossoverPoint1 = random.randint(0, len(p1first_layer_weights) - 1)

    o1first_layer_weights = []
    o2first_layer_weights = []

    for i in range(len(p1first_layer_weights)):
        
        o1 = np.concatenate((p1first_layer_weights[i][:crossoverPoint1],(p2first_layer_weights[i][crossoverPoint1:])))
        o2 = np.concatenate((p2first_layer_weights[i][:crossoverPoint1], p1first_layer_weights[i][crossoverPoint1:]))

        o1first_layer_weights.append(o1)
        o2first_layer_weights.append(o2)

    crossoverPoint2 = random.randint(0, len(p1first_layer_bias) - 1)
   
    o1first_layer_bias = []
    o2first_layer_bias = []

    for i in range(len(p1first_layer_bias)):

        o1 = np.concatenate((p1first_layer_bias[i][:crossoverPoint2], p2first_layer_bias[i][crossoverPoint2:]))
        o2 = np.concatenate((p2first_layer_bias[i][:crossoverPoint2], p1first_layer_bias[i][crossoverPoint2:]))

        o1first_layer_bias.append(o1)
        o2first_layer_bias.append(o2)

    crossoverPoint3 = random.randint(0, len(p1second_layer_weights) - 1)
   
    o1second_layer_weights = []
    o2second_layer_weights = []

    for i in range(len(p1second_layer_weights)):

        o1 = np.concatenate((p1second_layer_weights[i][:crossoverPoint3], p2second_layer_weights[i][crossoverPoint3:]))
        o2 = np.concatenate((p2second_layer_weights[i][:crossoverPoint3], p1second_layer_weights[i][crossoverPoint3:]))

        o1second_layer_weights.append(o1)
        o2second_layer_weights.append(o2)

    crossoverPoint4 = random.randint(0, len(p1second_layer_bias) - 1)

    o1second_layer_bias = []
    o2second_layer_bias = []

    for i in range(len(p1second_layer_bias)):

        o1 = np.concatenate((p1second_layer_bias[i][:crossoverPoint4], p2second_layer_bias[i][crossoverPoint4:]))
        o2 = np.concatenate((p2second_layer_bias[i][:crossoverPoint4], p1second_layer_bias[i][crossoverPoint4:]))

        o1second_layer_bias.append(o1)
        o2second_layer_bias.append(o2)

    crossoverPoint5 = random.randint(0, len(p1third_layer_weights) - 1)

    o1third_layer_weights = []
    o2third_layer_weights = []

    for i in range(len(p1third_layer_weights)):

        o1 = np.concatenate((p1third_layer_weights[i][:crossoverPoint5], p2third_layer_weights[i][crossoverPoint5:]))
        o2 = np.concatenate((p2third_layer_weights[i][:crossoverPoint5], p1third_layer_weights[i][crossoverPoint5:]))

        o1third_layer_weights.append(o1)
        o2third_layer_weights.append(o2)

    crossoverPoint6 = random.randint(0, len(p1third_layer_bias) - 1)

    o1third_layer_bias = []
    o2third_layer_bias = []

    for i in range(len(p1third_layer_bias)):

        o1 = np.concatenate((p1third_layer_bias[0][:crossoverPoint6], p2third_layer_bias[0][crossoverPoint6:]))
        o2 = np.concatenate((p2third_layer_bias[0][:crossoverPoint6], p1third_layer_bias[0][crossoverPoint6:]))

        o1third_layer_bias.append(o1)
        o2third_layer_bias.append(o2)

    offspring1 = []
    offspring1.append(o1first_layer_bias)
    offspring1.append(o1second_layer_bias)
    offspring1.append(o1third_layer_bias)

    offspring1.append(o1first_layer_weights)
    offspring1.append(o1second_layer_weights)
    offspring1.append(o1third_layer_weights)

    offspring2 = []
    offspring2.append(o2first_layer_bias)
    offspring2.append(o2second_layer_bias)
    offspring2.append(o2third_layer_bias)

    offspring2.append(o2first_layer_weights)
    offspring2.append(o2second_layer_weights)
    offspring2.append(o2third_layer_weights)

    return offspring1, offspring2

In [12]:
def uniform_crossover(parent1, parent2):

    p1_weights_biases = parent1.getWeights()
    p2_weights_biases = parent2.getWeights()

    o1_weights_biases = []
    o2_weights_biases = []

    for p1_layer, p2_layer in zip(p1_weights_biases, p2_weights_biases):
        mask = np.random.randint(0, 2, p1_layer.shape).astype(bool)
        o1_layer = np.where(mask, p1_layer, p2_layer)
        o2_layer = np.where(mask, p2_layer, p1_layer)

        o1_weights_biases.append(o1_layer)
        o2_weights_biases.append(o2_layer)

    #offspring1 = o1_weights_biases
    #offspring2 = o2_weights_biases

    p1first_layer_weights, p1first_layer_bias, p1second_layer_weights, p1second_layer_bias, p1third_layer_weights,  p1third_layer_bias = o1_weights_biases
    p2first_layer_weights, p2first_layer_bias, p2second_layer_weights, p2second_layer_bias, p2third_layer_weights,  p2third_layer_bias = o2_weights_biases
    
    offspring1 = []
    offspring1.append(p1first_layer_bias)
    offspring1.append(p1second_layer_bias)
    offspring1.append(p1third_layer_bias)

    offspring1.append(p1first_layer_weights)
    offspring1.append(p1second_layer_weights)
    offspring1.append(p1third_layer_weights)

    offspring2 = []
    offspring2.append(p2first_layer_bias)
    offspring2.append(p2second_layer_bias)
    offspring2.append(p2third_layer_bias)

    offspring2.append(p2first_layer_weights)
    offspring2.append(p2second_layer_weights)
    offspring2.append(p2third_layer_weights)

    return offspring1, offspring2


In [13]:
def mutationNew(offspring1, offspring2, mutationRate):

    o1first_layer_bias, o1second_layer_bias, o1third_layer_bias, o1first_layer_weights, o1second_layer_weights, o1third_layer_weights = offspring1
    o2first_layer_bias, o2second_layer_bias, o2third_layer_bias, o2first_layer_weights, o2second_layer_weights, o2third_layer_weights = offspring2
    
    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1first_layer_bias[0]) - 1, size=10)

        for i in range(10):
             o1first_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2first_layer_bias[0]) - 1, size=10)


        for i in range(10):
            o2first_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1second_layer_bias[0]) - 1, size=10)

        for i in range(10):
             o1second_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2second_layer_bias[0]) - 1, size=10)


        for i in range(10):
            o2second_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1third_layer_bias[0]) - 1, size=10)

        for i in range(10):
             o1third_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)

        #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2third_layer_bias[0]) - 1, size=10)


        for i in range(10):
            o2third_layer_bias[0][rand_nums[i]] = np.random.normal(0, 1)
            
        #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1first_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1first_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o1first_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2first_layer_weights[0]) - 1, size=10)


            for j in range(10):
                o2first_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1second_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1second_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o1second_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2second_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o2second_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1third_layer_weights) - 1):
         if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1third_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o1third_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)

            #o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2third_layer_weights[0]) - 1, size=10)

            for j in range(10):
                o2third_layer_weights[i][rand_nums[j]] = np.random.normal(0, 1)
                
            #o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
            #o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
    mutatedOffspring1 = []
    mutatedOffspring1.append(o1first_layer_bias)
    mutatedOffspring1.append(o1second_layer_bias)
    mutatedOffspring1.append(o1third_layer_bias)

    mutatedOffspring1.append(o1first_layer_weights)
    mutatedOffspring1.append(o1second_layer_weights)
    mutatedOffspring1.append(o1third_layer_weights)

    mutatedOffspring2 = []
    mutatedOffspring2.append(o2first_layer_bias)
    mutatedOffspring2.append(o2second_layer_bias)
    mutatedOffspring2.append(o2third_layer_bias)

    mutatedOffspring2.append(o2first_layer_weights)
    mutatedOffspring2.append(o2second_layer_weights)
    mutatedOffspring2.append(o2third_layer_weights)

    return mutatedOffspring1, mutatedOffspring2

In [14]:
def mutation(offspring1, offspring2, mutationRate):

    o1first_layer_bias, o1second_layer_bias, o1third_layer_bias, o1first_layer_weights, o1second_layer_weights, o1third_layer_weights = offspring1
    o2first_layer_bias, o2second_layer_bias, o2third_layer_bias, o2first_layer_weights, o2second_layer_weights, o2third_layer_weights = offspring2
    
    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1first_layer_bias[0]) - 1, size=2)
        o1first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
        

        rand_nums = np.random.randint(0, len(o2first_layer_bias[0]) - 1, size=2)
        o2first_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2first_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1second_layer_bias[0]) - 1, size=2)
        o1second_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1second_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)
        
        rand_nums = np.random.randint(0, len(o2second_layer_bias[0]) - 1, size=2)
        o2second_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2second_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    if random.uniform(0, 1) < mutationRate:
        rand_nums = np.random.randint(0, len(o1third_layer_bias[0]) - 1, size=2)
        o1third_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o1third_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

        rand_nums = np.random.randint(0, len(o2third_layer_bias[0]) - 1, size=2)
        o2third_layer_bias[0][rand_nums[0]] = np.random.normal(0, 1)
        o2third_layer_bias[0][rand_nums[1]] = np.random.normal(0, 1)

    for i in range(len(o1first_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1first_layer_weights[0]) - 1, size=2)
            o1first_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1first_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2first_layer_weights[0]) - 1, size=2)
            o2first_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2first_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1second_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1second_layer_weights[0]) - 1, size=2)
            o1second_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1second_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2second_layer_weights[0]) - 1, size=2)
            o2second_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2second_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)


    for i in range(len(o1third_layer_weights) - 1):
        if random.uniform(0, 1) < mutationRate:
            rand_nums = np.random.randint(0, len(o1third_layer_weights[0]) - 1, size=2)
            o1third_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o1third_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

            rand_nums = np.random.randint(0, len(o2third_layer_weights[0]) - 1, size=2)
            o2third_layer_weights[i][rand_nums[0]] = np.random.normal(0, 1)
            o2third_layer_weights[i][rand_nums[1]] = np.random.normal(0, 1)

    mutatedOffspring1 = []
    mutatedOffspring1.append(o1first_layer_bias)
    mutatedOffspring1.append(o1second_layer_bias)
    mutatedOffspring1.append(o1third_layer_bias)

    mutatedOffspring1.append(o1first_layer_weights)
    mutatedOffspring1.append(o1second_layer_weights)
    mutatedOffspring1.append(o1third_layer_weights)

    mutatedOffspring2 = []
    mutatedOffspring2.append(o2first_layer_bias)
    mutatedOffspring2.append(o2second_layer_bias)
    mutatedOffspring2.append(o2third_layer_bias)

    mutatedOffspring2.append(o2first_layer_weights)
    mutatedOffspring2.append(o2second_layer_weights)
    mutatedOffspring2.append(o2third_layer_weights)

    return mutatedOffspring1, mutatedOffspring2

In [15]:
def calculateFitness(population):

    for i in range(len(population) - 1):

        player1 = population[i]

        challengers = population[:i] +  population[i+1:]

        randomPlayers = np.random.choice(challengers, size=5, replace=False)

        for player2 in randomPlayers:
            obj = Game(player1, player2)
            new_board, winner, best_player = AI_vs_AI(player1, player2)

            if winner=="draw":
                player1.score +=1
                player2.score +=1
            else:
                best_player.score +=2

                if winner == 'red':
                    player1.score -= 1
                else:
                    player2.score -= 1

            #print("game complete")

In [16]:
def calculateFitnessOffspring(offspring1, offspring2, population):
    
    players = [offspring1, offspring2]

    for player1 in players:

        randomPlayers = np.random.choice(population, size=5, replace=False)
    
        for player2 in randomPlayers:
            obj = Game(offspring1, player2)
            new_board, winner, best_player = AI_vs_AI(offspring1, player2)

            if winner=="draw":
                offspring1.score +=1
                player2.score +=1
            else:
                best_player.score +=2

                if winner == 'red':
                    player2.score -= 1
                else:
                    offspring1.score -= 1

                
    return population + players


    

In [17]:
def fitnessForBinaryTournament_2(parent_1, parent_2, chromosomes):
    
    newchromosomes = chromosomes.copy()

    #newchromosomes.remove(parent_1)
    #newchromosomes.remove(parent_2)

    parent1Score = 0
    parent2Score = 0
    
    print("chromosomes left", len(newchromosomes))

   
    randomPlayers = np.random.choice(newchromosomes, size=2, replace=False)
    
    for player2 in randomPlayers:
        print("match start")
        obj = Game(parent_1, player2)
        new_board, winner, best_player = AI_vs_AI(parent_1, player2)

        # if winner=="draw":
        #     parent1Score += 1

        #     player2.draw += 1
        #     parent_1.draw += 1

        #     parent_1.score += 1
        #     player2.score += 1

        # else:
        if winner == 'white':
            # print("bnum: ",best_player.number)
            # parent1Score -= 1
            # parent_1.loss += 1
            # player2.win += 1
            # parent_1.score -= 1
            # player2.score += 2
            parent_1.win += 1
            player2.loss += 1

        else:
            # parent1Score += 2
            # parent_1.win += 1
            # player2.loss += 1
            # parent_1.score += 2
            # player2.score -= 1
            player2.win += 1
            parent_1.loss += 1
        
        player2.score = player2.win - player2.loss
        parent_1.score = parent_1.win - parent_1.loss
        parent1Score = parent_1.score

    for player2 in randomPlayers:
        obj = Game(parent_2, player2)
        new_board, winner, best_player = AI_vs_AI(parent_2, player2)

        # if winner=="draw":
        #     parent2Score += 1

        #     parent_2.draw += 1
        #     player2.draw += 1

        #     parent_2.score += 1
        #     player2.score += 1

        # else:
        if winner == 'white':
            # print("winner", winner, best_player)
            
            # print(parent_2)
            # parent2Score -= 1
            # parent_2.loss += 1
            # player2.win += 1
            # parent_2.score -= 1
            # player2.score += 2
            parent_2.win += 1
            player2.loss += 1
        else:
            # parent2Score += 2
            # player2.loss += 1
            # parent_2.win += 1
            # parent_2.score += 2
            # player2.score -= 1
            player2.win += 1
            parent_2.loss += 1
        
        player2.score = player2.win - player2.loss
        parent_2.score = parent_2.win - parent_2.loss
        parent2Score = parent_2.score
            


    if parent2Score > parent1Score:
        return parent_2
    else:
        return parent_1

    

In [18]:
def survivorSelection(newPopulation, type, totalnum):

    finalPopulation = []
        
    if type == "truncation":

        tup = []
        for i in newPopulation:
            tup.append((i.score, i))
        
        sortedlist = sorted(tup, key=lambda x: x[0], reverse=True)

        for pop in sortedlist:
            finalPopulation.append(pop[1])

        finalPopulation =  finalPopulation[:10]
    
    elif type == "fps":
        total_fitness = sum(chromosome.score for chromosome in newPopulation)
        chromosome_probabilities = [chromosome.fitness/total_fitness for chromosome in newPopulation]
        for i in range(10):
            finalPopulation.append(np.random.choice(newPopulation, p=chromosome_probabilities))


    elif type == "binary_tournament":
         
        for i in range(totalnum):
        # Choose two random parents from the population
            index = np.random.choice(newPopulation, 2, replace=False) 

            parent1 = index[0]
            parent2 = index[1]

            # Choose the fittest chromosome as the first parent
            if parent1.score > parent2.score:
                finalPopulation.append(parent1)
            else:
                finalPopulation.append(parent2)

    elif type == "tournament":
       
       nnewPopulation = newPopulation.copy()

       print("lenght", len(newPopulation))

       for i in range(totalnum):
           
           index = np.random.choice(nnewPopulation, 2, replace=False) 

           parent1 = index[0]
           parent2 = index[1]
           
           #parent1 = random.choice(newPopulation)
           #parent2 = random.choice(newPopulation)

           chosen = fitnessForBinaryTournament_2(parent1, parent2, nnewPopulation)
           
           finalPopulation.append(chosen)
           nnewPopulation.remove(chosen)
            
           print(len(finalPopulation))
    
    return finalPopulation

In [33]:
def NueroEvolution2(population, generations):

    best_overall_player_score = 0

    chromosomes = []

    #Initialise random population
    for i in range(population):
        chromosomes.append(evolutionary_player(i))
    
    count = 25
    
    for n in range(generations):
        
        print("Generation: ", n)

        parent1, parent2 = parentSelection(chromosomes, "binary_tournament")

        print("Parent 1", parent1)
        print("Parent 2", parent2)

        offspring1, offspring2 = uniform_crossover(parent1, parent2)  
        print("Crossover complete")

        offspring1AfterMutation, offspring2AfterMutation = mutationNew(offspring1, offspring2, mutationRate=0.1)

        parent1player, parent2player = createNeuralNetwork(offspring1AfterMutation, offspring2AfterMutation, count)
        print("Mutation complete")

        newPopulation = chromosomes + [parent1player, parent2player]
        
        #newPopulation = calculateFitnessOffspring(parent1player, parent2player, chromosomes)

        #calculateFitness(newPopulation)
        print("Fitness Calculation complete")

        chromosomes = survivorSelection(newPopulation, "truncation", population)

        
        
        print("Generation Scores")
        for i in chromosomes:
            
            print("wins:", i.win, "losses:", i.loss, "draws:", i.draw, "score", i.score)
        
    

        count += 2

    max_fit = 0
    player = None
    for i in chromosomes:
        print(i)
        if i.score > max_fit:
            max_fit = i.score
            player = i
    return player

    

In [34]:
player2 = NueroEvolution2(12, 5)

Generation:  0
Tournament for Parent 1
match start
match start
match start
match start
match start
match start
Parent 1 <__main__.Evol_Player object at 0x0000016B7CDCE3B0>
Parent 2 <__main__.Evol_Player object at 0x0000016B7D558700>
Crossover complete
Mutation complete
Fitness Calculation complete
Generation Scores
wins: 4 losses: 0 draws: 0 score 4
wins: 2 losses: 0 draws: 0 score 2
wins: 2 losses: 0 draws: 0 score 2
wins: 0 losses: 0 draws: 0 score 0
wins: 0 losses: 0 draws: 0 score 0
wins: 0 losses: 0 draws: 0 score 0
wins: 1 losses: 1 draws: 0 score 0
wins: 0 losses: 0 draws: 0 score 0
wins: 0 losses: 0 draws: 0 score 0
wins: 0 losses: 0 draws: 0 score 0
Generation:  1
Tournament for Parent 1
match start
match start
match start
match start
match start
match start
Parent 1 <__main__.Evol_Player object at 0x0000016B7CDCC040>
Parent 2 <__main__.Evol_Player object at 0x0000016B7D5583D0>
Crossover complete
Mutation complete
Fitness Calculation complete
Generation Scores
wins: 4 losses: 

In [35]:
import sys
Sample_array_2 = np.arange(100)
np.set_printoptions(threshold=sys.maxsize)
print(player2.first_layer_weights)

[[-9.69265046e-01 -4.98168368e-01 -6.67685943e-01  4.48581290e-01
  -3.41281152e-01  6.94004360e-01  6.14296069e-01 -1.02347458e-01
   1.58466900e-01 -1.09662384e-01  1.06936624e-01 -4.64349554e-02
   3.86711255e+00 -4.18765292e-01  7.36664844e-01  1.10342845e+00
   9.65336093e-01 -2.45872562e+00  5.73693158e-01 -1.45311671e+00
  -6.74454093e-01 -6.42142418e-01  8.61972655e-01 -1.17888858e+00
   1.66971335e+00 -3.04485923e-01  1.02282704e+00  3.59004861e-01
   9.44000713e-01 -1.08012468e+00  1.19699544e+00  2.11857079e-02
  -5.12286321e-01 -9.41911776e-02  1.17379571e+00 -2.44977327e-01
  -1.16761400e+00 -2.29159339e-01 -8.10052273e-01 -1.09612309e+00
   1.47593829e+00  5.63108864e-01 -5.42279312e-02 -1.69009273e-01
   4.22183605e-01 -5.17960415e-01 -2.75072871e-02  8.15805530e-04
  -3.02788816e-02  1.42086326e+00  5.95263171e-01  1.98520714e+00
   5.85769566e-01  2.17791672e-02 -8.94445079e-01  1.10013187e+00
   1.14214911e+00  1.23930373e+00  1.09832675e+00 -1.45408332e-01
  -7.98230

In [36]:
print(player2.second_layer_weights)

[[ 5.48061868e-01 -5.83050248e-01 -6.70093106e-01  1.02406384e+00
   7.55442763e-01 -1.61725233e-01  7.17694354e-01 -1.30670637e+00
  -1.13838248e+00 -1.27632240e+00  1.18303808e+00 -1.10038106e+00
  -7.21898846e-01  1.12569564e+00 -5.62474853e-01 -7.53049714e-01
  -6.26879301e-01 -1.51085799e+00 -2.96551662e-01  1.29803859e+00
   6.41615013e-01  3.47613421e-01  6.58515131e-02  4.38420467e-01
   1.37092196e+00  1.20426707e+00 -9.24182685e-01 -8.59012869e-01
  -1.25678712e-02 -1.04780892e+00  1.21158024e-01  1.23047311e+00
  -1.69932616e+00 -2.02180635e-01  3.79349481e-01  3.55917811e-01
   2.14667612e+00 -2.86499371e-01 -1.76824400e+00 -3.08680434e-01]
 [ 1.28364548e+00  3.62955942e-01 -4.57877228e-01 -1.27049165e+00
   4.54742687e-02  4.21733932e-01 -1.92890070e+00 -2.60480300e-01
  -1.42145015e+00  5.18123631e-01  7.48340621e-01 -1.14605056e+00
  -1.52692211e+00 -6.43206770e-01  8.93183657e-01 -1.26959396e+00
  -1.60660941e+00  5.93584391e-01 -1.73159411e-01 -1.23258028e+00
  -2.1515

In [37]:
print(player2.third_layer_weights)

[[ 1.44905694e+00 -4.05143552e-01 -4.35155603e-01  3.36199172e-01
   3.94678044e-01  4.39729111e-01 -1.65107485e+00  1.67169084e-01
  -1.71438400e+00  2.10342483e-01]
 [ 7.94102822e-01 -1.28982709e-01  1.23687949e+00  6.09064132e-02
   2.80187554e-01  9.89171553e-01 -1.49604139e+00  1.26001255e+00
  -4.35710686e-01  4.60079282e-01]
 [ 2.23114372e+00  2.14548445e-01 -4.36077324e-01 -4.65850734e-01
  -1.50684951e+00 -3.44751068e-01  1.15494593e+00 -3.86392006e-01
  -4.63279636e-01 -3.85263969e-01]
 [ 1.42401853e+00  8.49367474e-01 -1.96766042e-01 -6.61151291e-01
   1.82953480e-01  4.27240553e-01 -2.66632813e-01 -1.70977389e-01
  -1.35159842e+00  4.77356611e-01]
 [ 4.86612629e-01 -4.02721900e-01 -1.70532749e+00 -1.06458505e+00
   5.87232166e-01 -1.44015767e+00  5.67626305e-01 -4.24133120e-01
  -6.49689678e-01  7.79240319e-01]
 [ 4.09446684e-01  5.30604757e-01 -1.71122367e-01 -7.22581935e-01
   1.41369503e+00  3.62955336e-01 -1.51882966e-01  2.15688401e-01
  -4.27872980e-01  1.38834303e-02

In [39]:
print(player2.first_layer_bias)

[[ 0.95098685  0.44261623 -0.09580206  1.1526399  -0.77702503 -0.71019926
   0.13720668  0.8358332  -1.1440471  -0.07205106  1.9652365  -3.37758446
   0.32848703 -0.76980849 -0.77731443 -0.81018664 -0.69872791  0.48251961
  -0.94725728 -0.45234327  1.23277877 -0.89298495  0.20559404  0.54812971
   0.44785256 -1.1017909  -0.24201754 -2.58399422  1.10756233 -1.44422914
  -0.33402373 -0.68038081 -0.16649911 -0.52483346  2.92637354 -0.32886684
   1.08464377 -0.47809224  0.42353006 -0.82524478  0.99247839  1.74092753
   0.482346    0.73575284  1.99812506 -1.10786363 -0.6551891   0.99104596
  -1.6687693  -0.43691504  0.19635909  0.14635761  0.53879895  0.55806108
   0.66850016 -0.90556539  1.25373114  0.47966459 -0.37281566  2.18534884
  -0.16481584 -1.6209085   0.68435085 -0.94165599 -2.20237848 -0.55181255
  -0.92913885  0.36870132 -0.02772776  0.46068861 -1.45472359  0.11901091
   0.25481504  0.66588601 -1.41668537 -2.56269308 -0.09942033 -0.31134997
  -1.02267494  1.2923919  -0.02434051 

In [40]:
print(player2.second_layer_bias)

[[ 1.24469719  1.05540743 -0.45506712  0.89425212 -0.41633673 -0.16539111
  -1.08614949 -1.17900739 -0.66157281  1.31828514 -0.41288425  1.89857618
  -0.83104567  0.66689615 -1.14226178 -2.56298425  1.41460336  0.23108079
  -0.22096129  0.97427872 -0.26976058  1.46030125 -0.46789434 -0.97357057
  -0.87406155  0.89376174 -0.04286543  1.10525214 -1.47160591 -0.81272186
  -1.01248289 -1.14863827  0.51822044 -0.86685929 -0.57117637  0.35182218
   1.19783067  1.14152316 -1.9077221   0.05173622]]


In [41]:
print(player2.third_layer_bias)

[[-1.16222936  0.38913643  1.67924833  0.16667533 -0.95128671 -0.61405107
  -0.17323778 -1.17270907 -1.04534962  0.07191034]]


In [1]:
new_board, winner, best_player = human_AI(player2)
#new_board, winner, best_player = AI_vs_AI()

for i in new_board.board:
    print(i)
for i in new_board.board:
    for piece in i:
        if piece !=0:
            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)
print("WINNER COLOR:")
print(winner)
print("WINNER PLAYER NUM:")
print(best_player.number)
print("best first layer weights:")
print(best_player.first_layer_weights)
print("best first layer bias:")
print(best_player.first_layer_bias)
print("best second layer weights:")
print(best_player.second_layer_weights)
print("best second layer bias:")
print(best_player.second_layer_bias)
print("best third layer weights:")
print(best_player.third_layer_weights)
print("best third layer bias:")

NameError: name 'human_AI' is not defined

In [ ]:
new_board, winner, best_player = human_AI(player2)
#new_board, winner, best_player = AI_vs_AI()

for i in new_board.board:
    print(i)
for i in new_board.board:
    for piece in i:
        if piece !=0:
            print("PIECE COLOR: ", piece.color, " IS KING?: ", piece.king)
print("WINNER COLOR:")
print(winner)
print("WINNER PLAYER NUM:")
print(best_player.number)
print("best first layer weights:")
print(best_player.first_layer_weights)
print("best first layer bias:")
print(best_player.first_layer_bias)
print("best second layer weights:")
print(best_player.second_layer_weights)
print("best second layer bias:")
print(best_player.second_layer_bias)
print("best third layer weights:")
print(best_player.third_layer_weights)
print("best third layer bias:")